In [1]:
import tweepy
import datetime
import csv, string, re
import pandas as pd
import nltk

In [2]:
api_key = ""
api_key_secret = ""
access_token = ""
access_token_secret = ""

In [3]:
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [4]:
csvFile = open('twitter2.csv', 'w', encoding='utf-8')
csvWriter = csv.writer(csvFile)
schema = ['id_str', 'user', 'tweet','created']
csvWriter.writerow([g for g in schema]) 
for tweet in tweepy.Cursor(api.search_tweets, q='presiden+3+periode -filter:retweets', tweet_mode='extended', lang="id").items(2000):
    id_str = tweet.id_str
    user = tweet.user.name
    text = tweet.full_text
    created = tweet.created_at.strftime("%m/%d/%Y %H:%M:%S")
    csvWriter.writerow([id_str, user, text.encode('utf-8'), created])
csvWriter = csv.writer(csvFile)
csvFile.close()

In [12]:
data = pd.read_csv("twitter2.csv", encoding="utf-8")
data.head()

,id_str,user,tweet,created
0,1515126657757196299,REI & CO LAWYER OFFICE,b'Pimpinan DPRD Wajo Antarkan Aspirasi Penolak...,04/16/2022 00:35:19
1,1515118363802206211,HermanSL,b'@almerdiarsy @IndraVision3 @_ekokuntadhi Kok...,04/16/2022 00:02:22
2,1515113819080560642,Peter Gunawan,b'@jokowi @Kemenkumham_RI @PolhukamRI Gak ada ...,04/15/2022 23:44:18
3,1515113386006413313,Kerja Keras,b'Soal Penundaan pemilu &amp; jabatan presiden...,04/15/2022 23:42:35
4,1515101461268819969,Naura,"b'@geloraco Bermain kata. Pemilu tunda, presid...",04/15/2022 22:55:12


In [13]:
# casefolding
data['casefolding'] = data['tweet'].apply(lambda x: x.lower())
data = data [['id_str','tweet', 'casefolding']]
data.head()

,id_str,tweet,casefolding
0,1515126657757196299,b'Pimpinan DPRD Wajo Antarkan Aspirasi Penolak...,b'pimpinan dprd wajo antarkan aspirasi penolak...
1,1515118363802206211,b'@almerdiarsy @IndraVision3 @_ekokuntadhi Kok...,b'@almerdiarsy @indravision3 @_ekokuntadhi kok...
2,1515113819080560642,b'@jokowi @Kemenkumham_RI @PolhukamRI Gak ada ...,b'@jokowi @kemenkumham_ri @polhukamri gak ada ...
3,1515113386006413313,b'Soal Penundaan pemilu &amp; jabatan presiden...,b'soal penundaan pemilu &amp; jabatan presiden...
4,1515101461268819969,"b'@geloraco Bermain kata. Pemilu tunda, presid...","b'@geloraco bermain kata. pemilu tunda, presid..."


In [14]:
# cleaning
string.punctuation

def cleaning(tweet):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        u"\U00002500-\U00002BEF"
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"
        u"\u3030"
                      "]+", re.UNICODE)
    tweet = re.sub(emoj, '', tweet)
    tweet = re.sub('b\'', '', tweet)
    tweet = tweet[:-1]
    tweet = re.sub('@[\w]+','', tweet)
    tweet = re.sub(r'http\S+', '', tweet)
    tweet2 = "".join([i for i in tweet if i not in string.punctuation])
    tweet3 = ''.join([i for i in tweet2 if not i.isdigit()])
    return tweet3

data['cleaning'] = data['casefolding'].apply(lambda x:cleaning(x))
data

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


,id_str,tweet,casefolding,cleaning
0,1515126657757196299,b'Pimpinan DPRD Wajo Antarkan Aspirasi Penolak...,b'pimpinan dprd wajo antarkan aspirasi penolak...,pimpinan dprd wajo antarkan aspirasi penolakan...
1,1515118363802206211,b'@almerdiarsy @IndraVision3 @_ekokuntadhi Kok...,b'@almerdiarsy @indravision3 @_ekokuntadhi kok...,kok ga tau kalo kewenangan periode urusan ...
2,1515113819080560642,b'@jokowi @Kemenkumham_RI @PolhukamRI Gak ada ...,b'@jokowi @kemenkumham_ri @polhukamri gak ada ...,gak ada berita ttg mutasi nih tumben biasan...
3,1515113386006413313,b'Soal Penundaan pemilu &amp; jabatan presiden...,b'soal penundaan pemilu &amp; jabatan presiden...,soal penundaan pemilu amp jabatan presiden pe...
4,1515101461268819969,"b'@geloraco Bermain kata. Pemilu tunda, presid...","b'@geloraco bermain kata. pemilu tunda, presid...",bermain kata pemilu tunda presiden lanjut ber...
...,...,...,...,...
95,1514887907370467328,b'@KangUtang04 @MatahariTimur17 @I205mdr____ @...,b'@kangutang04 @mataharitimur17 @i205mdr____ @...,njokowi luhut tito teroris kon...
96,1514885059886534658,b'@MustofaNahra_ID \xc2\xb0\xc2\xb0\nDi Sri La...,b'@mustofanahra_id \xc2\xb0\xc2\xb0\ndi sri la...,xcxbxcxbndi sri langka setelah presiden nya m...
97,1514885006014881796,b'@EdiMahaMG \xc2\xb0\xc2\xb0\nDi Sri Langka s...,b'@edimahamg \xc2\xb0\xc2\xb0\ndi sri langka s...,xcxbxcxbndi sri langka setelah presiden nya m...
98,1514883617004359680,b'@KontraS Dia yg bikin isu penundaan &amp; 3 ...,b'@kontras dia yg bikin isu penundaan &amp; 3 ...,dia yg bikin isu penundaan amp periode presi...


In [15]:
# tokenization
def tokenization(text):
#     tokens = text.split()
    tokens = nltk.tokenize.word_tokenize(text)
    return tokens
data['tokenizing'] = data['cleaning'].apply(lambda x: tokenization(x))
data.head()

,id_str,tweet,casefolding,cleaning,tokenizing
0,1515126657757196299,b'Pimpinan DPRD Wajo Antarkan Aspirasi Penolak...,b'pimpinan dprd wajo antarkan aspirasi penolak...,pimpinan dprd wajo antarkan aspirasi penolakan...,"[pimpinan, dprd, wajo, antarkan, aspirasi, pen..."
1,1515118363802206211,b'@almerdiarsy @IndraVision3 @_ekokuntadhi Kok...,b'@almerdiarsy @indravision3 @_ekokuntadhi kok...,kok ga tau kalo kewenangan periode urusan ...,"[kok, ga, tau, kalo, kewenangan, periode, urus..."
2,1515113819080560642,b'@jokowi @Kemenkumham_RI @PolhukamRI Gak ada ...,b'@jokowi @kemenkumham_ri @polhukamri gak ada ...,gak ada berita ttg mutasi nih tumben biasan...,"[gak, ada, berita, ttg, mutasi, nih, tumben, b..."
3,1515113386006413313,b'Soal Penundaan pemilu &amp; jabatan presiden...,b'soal penundaan pemilu &amp; jabatan presiden...,soal penundaan pemilu amp jabatan presiden pe...,"[soal, penundaan, pemilu, amp, jabatan, presid..."
4,1515101461268819969,"b'@geloraco Bermain kata. Pemilu tunda, presid...","b'@geloraco bermain kata. pemilu tunda, presid...",bermain kata pemilu tunda presiden lanjut ber...,"[bermain, kata, pemilu, tunda, presiden, lanju..."


In [17]:
stop_factory = nltk.corpus.stopwords.words('indonesian')

more_stopword = ['xcxbxcxbndi', 'si', 'sih', 'gak', 'ga', 'ttg', 'yg', 'nya', 'amp', 'kalo', 'nih', 'nn', 'tau']
stop_words = stop_factory + more_stopword

['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasanya', 'baik', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai', 'berdatangan', 'beri', 'berikan', 'berikut', 'berikutnya', 'berjumlah', 'berkali-kali', 'berkata', 'berkehendak', 'berkeinginan'

In [18]:
# filtering
def filtering(text):
    tokens = [w for w in text if not w in stop_words]
    return tokens
data['nostopword'] = data['tokenizing'].apply(lambda x: filtering(x))
data

,id_str,tweet,casefolding,cleaning,tokenizing,nostopword
0,1515126657757196299,b'Pimpinan DPRD Wajo Antarkan Aspirasi Penolak...,b'pimpinan dprd wajo antarkan aspirasi penolak...,pimpinan dprd wajo antarkan aspirasi penolakan...,"[pimpinan, dprd, wajo, antarkan, aspirasi, pen...","[pimpinan, dprd, wajo, antarkan, aspirasi, pen..."
1,1515118363802206211,b'@almerdiarsy @IndraVision3 @_ekokuntadhi Kok...,b'@almerdiarsy @indravision3 @_ekokuntadhi kok...,kok ga tau kalo kewenangan periode urusan ...,"[kok, ga, tau, kalo, kewenangan, periode, urus...","[kewenangan, periode, urusan, dpr, demo, presi..."
2,1515113819080560642,b'@jokowi @Kemenkumham_RI @PolhukamRI Gak ada ...,b'@jokowi @kemenkumham_ri @polhukamri gak ada ...,gak ada berita ttg mutasi nih tumben biasan...,"[gak, ada, berita, ttg, mutasi, nih, tumben, b...","[berita, mutasi, tumben, suami, dimutasi]"
3,1515113386006413313,b'Soal Penundaan pemilu &amp; jabatan presiden...,b'soal penundaan pemilu &amp; jabatan presiden...,soal penundaan pemilu amp jabatan presiden pe...,"[soal, penundaan, pemilu, amp, jabatan, presid...","[penundaan, pemilu, jabatan, presiden, periode]"
4,1515101461268819969,"b'@geloraco Bermain kata. Pemilu tunda, presid...","b'@geloraco bermain kata. pemilu tunda, presid...",bermain kata pemilu tunda presiden lanjut ber...,"[bermain, kata, pemilu, tunda, presiden, lanju...","[bermain, pemilu, tunda, presiden, langsung, p..."
...,...,...,...,...,...,...
95,1514887907370467328,b'@KangUtang04 @MatahariTimur17 @I205mdr____ @...,b'@kangutang04 @mataharitimur17 @i205mdr____ @...,njokowi luhut tito teroris kon...,"[njokowi, luhut, tito, teroris, konstitusinnpa...","[njokowi, luhut, tito, teroris, konstitusinnpa..."
96,1514885059886534658,b'@MustofaNahra_ID \xc2\xb0\xc2\xb0\nDi Sri La...,b'@mustofanahra_id \xc2\xb0\xc2\xb0\ndi sri la...,xcxbxcxbndi sri langka setelah presiden nya m...,"[xcxbxcxbndi, sri, langka, setelah, presiden, ...","[sri, langka, presiden, menumpuk, utang, kepin..."
97,1514885006014881796,b'@EdiMahaMG \xc2\xb0\xc2\xb0\nDi Sri Langka s...,b'@edimahamg \xc2\xb0\xc2\xb0\ndi sri langka s...,xcxbxcxbndi sri langka setelah presiden nya m...,"[xcxbxcxbndi, sri, langka, setelah, presiden, ...","[sri, langka, presiden, menumpuk, utang, kepin..."
98,1514883617004359680,b'@KontraS Dia yg bikin isu penundaan &amp; 3 ...,b'@kontras dia yg bikin isu penundaan &amp; 3 ...,dia yg bikin isu penundaan amp periode presi...,"[dia, yg, bikin, isu, penundaan, amp, periode,...","[bikin, isu, penundaan, periode, presiden, dem..."


In [19]:
# stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

exception = ['mentri', 'pemilu']
def stemming(text):
    tokens = []
    for lst in text:
        if(lst not in exception):
            hasil = stemmer.stem(lst)
            tokens.append(hasil)
        else:
            tokens.append(lst)
    return tokens
data['stemming'] = data['nostopword'].apply(lambda x: stemming(x))
data

,id_str,tweet,casefolding,cleaning,tokenizing,nostopword,stemming
0,1515126657757196299,b'Pimpinan DPRD Wajo Antarkan Aspirasi Penolak...,b'pimpinan dprd wajo antarkan aspirasi penolak...,pimpinan dprd wajo antarkan aspirasi penolakan...,"[pimpinan, dprd, wajo, antarkan, aspirasi, pen...","[pimpinan, dprd, wajo, antarkan, aspirasi, pen...","[pimpin, dprd, wajo, antar, aspirasi, tolak, p..."
1,1515118363802206211,b'@almerdiarsy @IndraVision3 @_ekokuntadhi Kok...,b'@almerdiarsy @indravision3 @_ekokuntadhi kok...,kok ga tau kalo kewenangan periode urusan ...,"[kok, ga, tau, kalo, kewenangan, periode, urus...","[kewenangan, periode, urusan, dpr, demo, presi...","[wenang, periode, urus, dpr, demo, presiden, m..."
2,1515113819080560642,b'@jokowi @Kemenkumham_RI @PolhukamRI Gak ada ...,b'@jokowi @kemenkumham_ri @polhukamri gak ada ...,gak ada berita ttg mutasi nih tumben biasan...,"[gak, ada, berita, ttg, mutasi, nih, tumben, b...","[berita, mutasi, tumben, suami, dimutasi]","[berita, mutasi, tumben, suami, mutasi]"
3,1515113386006413313,b'Soal Penundaan pemilu &amp; jabatan presiden...,b'soal penundaan pemilu &amp; jabatan presiden...,soal penundaan pemilu amp jabatan presiden pe...,"[soal, penundaan, pemilu, amp, jabatan, presid...","[penundaan, pemilu, jabatan, presiden, periode]","[tunda, pemilu, jabat, presiden, periode]"
4,1515101461268819969,"b'@geloraco Bermain kata. Pemilu tunda, presid...","b'@geloraco bermain kata. pemilu tunda, presid...",bermain kata pemilu tunda presiden lanjut ber...,"[bermain, kata, pemilu, tunda, presiden, lanju...","[bermain, pemilu, tunda, presiden, langsung, p...","[main, pemilu, tunda, presiden, langsung, peri..."
...,...,...,...,...,...,...,...
95,1514887907370467328,b'@KangUtang04 @MatahariTimur17 @I205mdr____ @...,b'@kangutang04 @mataharitimur17 @i205mdr____ @...,njokowi luhut tito teroris kon...,"[njokowi, luhut, tito, teroris, konstitusinnpa...","[njokowi, luhut, tito, teroris, konstitusinnpa...","[njokowi, luhut, tito, teroris, konstitusinnpa..."
96,1514885059886534658,b'@MustofaNahra_ID \xc2\xb0\xc2\xb0\nDi Sri La...,b'@mustofanahra_id \xc2\xb0\xc2\xb0\ndi sri la...,xcxbxcxbndi sri langka setelah presiden nya m...,"[xcxbxcxbndi, sri, langka, setelah, presiden, ...","[sri, langka, presiden, menumpuk, utang, kepin...","[sri, langka, presiden, tumpuk, utang, kepincu..."
97,1514885006014881796,b'@EdiMahaMG \xc2\xb0\xc2\xb0\nDi Sri Langka s...,b'@edimahamg \xc2\xb0\xc2\xb0\ndi sri langka s...,xcxbxcxbndi sri langka setelah presiden nya m...,"[xcxbxcxbndi, sri, langka, setelah, presiden, ...","[sri, langka, presiden, menumpuk, utang, kepin...","[sri, langka, presiden, tumpuk, utang, kepincu..."
98,1514883617004359680,b'@KontraS Dia yg bikin isu penundaan &amp; 3 ...,b'@kontras dia yg bikin isu penundaan &amp; 3 ...,dia yg bikin isu penundaan amp periode presi...,"[dia, yg, bikin, isu, penundaan, amp, periode,...","[bikin, isu, penundaan, periode, presiden, dem...","[bikin, isu, tunda, periode, presiden, demo, n..."


In [20]:
def freq(text):
    tokens = nltk.FreqDist(text).most_common()
    return tokens
data['freq'] = data['stemming'].apply(lambda x: freq(x))
data

,id_str,tweet,casefolding,cleaning,tokenizing,nostopword,stemming,freq
0,1515126657757196299,b'Pimpinan DPRD Wajo Antarkan Aspirasi Penolak...,b'pimpinan dprd wajo antarkan aspirasi penolak...,pimpinan dprd wajo antarkan aspirasi penolakan...,"[pimpinan, dprd, wajo, antarkan, aspirasi, pen...","[pimpinan, dprd, wajo, antarkan, aspirasi, pen...","[pimpin, dprd, wajo, antar, aspirasi, tolak, p...","[(pimpin, 1), (dprd, 1), (wajo, 1), (antar, 1)..."
1,1515118363802206211,b'@almerdiarsy @IndraVision3 @_ekokuntadhi Kok...,b'@almerdiarsy @indravision3 @_ekokuntadhi kok...,kok ga tau kalo kewenangan periode urusan ...,"[kok, ga, tau, kalo, kewenangan, periode, urus...","[kewenangan, periode, urusan, dpr, demo, presi...","[wenang, periode, urus, dpr, demo, presiden, m...","[(periode, 2), (putus, 2), (wenang, 1), (urus,..."
2,1515113819080560642,b'@jokowi @Kemenkumham_RI @PolhukamRI Gak ada ...,b'@jokowi @kemenkumham_ri @polhukamri gak ada ...,gak ada berita ttg mutasi nih tumben biasan...,"[gak, ada, berita, ttg, mutasi, nih, tumben, b...","[berita, mutasi, tumben, suami, dimutasi]","[berita, mutasi, tumben, suami, mutasi]","[(mutasi, 2), (berita, 1), (tumben, 1), (suami..."
3,1515113386006413313,b'Soal Penundaan pemilu &amp; jabatan presiden...,b'soal penundaan pemilu &amp; jabatan presiden...,soal penundaan pemilu amp jabatan presiden pe...,"[soal, penundaan, pemilu, amp, jabatan, presid...","[penundaan, pemilu, jabatan, presiden, periode]","[tunda, pemilu, jabat, presiden, periode]","[(tunda, 1), (pemilu, 1), (jabat, 1), (preside..."
4,1515101461268819969,"b'@geloraco Bermain kata. Pemilu tunda, presid...","b'@geloraco bermain kata. pemilu tunda, presid...",bermain kata pemilu tunda presiden lanjut ber...,"[bermain, kata, pemilu, tunda, presiden, lanju...","[bermain, pemilu, tunda, presiden, langsung, p...","[main, pemilu, tunda, presiden, langsung, peri...","[(main, 1), (pemilu, 1), (tunda, 1), (presiden..."
...,...,...,...,...,...,...,...,...
95,1514887907370467328,b'@KangUtang04 @MatahariTimur17 @I205mdr____ @...,b'@kangutang04 @mataharitimur17 @i205mdr____ @...,njokowi luhut tito teroris kon...,"[njokowi, luhut, tito, teroris, konstitusinnpa...","[njokowi, luhut, tito, teroris, konstitusinnpa...","[njokowi, luhut, tito, teroris, konstitusinnpa...","[(jabat, 2), (njokowi, 1), (luhut, 1), (tito, ..."
96,1514885059886534658,b'@MustofaNahra_ID \xc2\xb0\xc2\xb0\nDi Sri La...,b'@mustofanahra_id \xc2\xb0\xc2\xb0\ndi sri la...,xcxbxcxbndi sri langka setelah presiden nya m...,"[xcxbxcxbndi, sri, langka, setelah, presiden, ...","[sri, langka, presiden, menumpuk, utang, kepin...","[sri, langka, presiden, tumpuk, utang, kepincu...","[(sri, 1), (langka, 1), (presiden, 1), (tumpuk..."
97,1514885006014881796,b'@EdiMahaMG \xc2\xb0\xc2\xb0\nDi Sri Langka s...,b'@edimahamg \xc2\xb0\xc2\xb0\ndi sri langka s...,xcxbxcxbndi sri langka setelah presiden nya m...,"[xcxbxcxbndi, sri, langka, setelah, presiden, ...","[sri, langka, presiden, menumpuk, utang, kepin...","[sri, langka, presiden, tumpuk, utang, kepincu...","[(sri, 1), (langka, 1), (presiden, 1), (tumpuk..."
98,1514883617004359680,b'@KontraS Dia yg bikin isu penundaan &amp; 3 ...,b'@kontras dia yg bikin isu penundaan &amp; 3 ...,dia yg bikin isu penundaan amp periode presi...,"[dia, yg, bikin, isu, penundaan, amp, periode,...","[bikin, isu, penundaan, periode, presiden, dem...","[bikin, isu, tunda, periode, presiden, demo, n...","[(bikin, 1), (isu, 1), (tunda, 1), (periode, 1..."


In [21]:
i = 0
while i < 10:
    print("Tweet: "+data['tweet'][i])
    print(data['cleaning'][i])
    print(data['nostopword'][i])
    print(data['stemming'][i])
    print(data['freq'][i])
    print('\n')
    i += 1

Tweet: b'Pimpinan DPRD Wajo Antarkan Aspirasi Penolakan 3 Periode Jabatan\xc2\xa0Presiden https://t.co/lM6DsiFXlv'
pimpinan dprd wajo antarkan aspirasi penolakan  periode jabatanxcxapresiden 
['pimpinan', 'dprd', 'wajo', 'antarkan', 'aspirasi', 'penolakan', 'periode', 'jabatanxcxapresiden']
['pimpin', 'dprd', 'wajo', 'antar', 'aspirasi', 'tolak', 'periode', 'jabatanxcxapresiden']
[('pimpin', 1), ('dprd', 1), ('wajo', 1), ('antar', 1), ('aspirasi', 1), ('tolak', 1), ('periode', 1), ('jabatanxcxapresiden', 1)]


Tweet: b'@almerdiarsy @IndraVision3 @_ekokuntadhi Kok ga tau kalo kewenangan 3 periode urusan dpr? Malah demo ke presiden? Jangan2 mahasiswa ui pikir keputusan mentri bisa memutuskan 3 periode? Ga heran sih mahasewa jaman sekarang'
   kok ga tau kalo kewenangan  periode urusan dpr malah demo ke presiden jangan mahasiswa ui pikir keputusan mentri bisa memutuskan  periode ga heran sih mahasewa jaman sekarang
['kewenangan', 'periode', 'urusan', 'dpr', 'demo', 'presiden', 'mahasiswa'